In [0]:
# pip install pandas matplotlib python-dateutil

from __future__ import annotations
from dataclasses import dataclass
from typing import Any, Dict, Optional, Tuple
from dateutil.parser import parse as dtparse
import re
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from io import StringIO

# ----------------------------- Shared types ----------------------------------
@dataclass
class TaskResult:
    ok: bool
    data: Any = None
    info: str = ""
    artifacts: Dict[str, str] = None  # e.g., {"plot": "quarterly_plot.png"}

# ----------------------------- Tools (functions) -----------------------------
def query_database(quarter: str, year: int) -> pd.DataFrame:
    """
    Mock "database" query. Returns a DataFrame with transactions for the given quarter.
    For demo, we synthesize a small dataset across Q1..Q4.
    """
    # Fake transaction table
    csv = StringIO(
        """date,amount,category,region
2024-10-03,12000,Subscriptions,NA
2024-10-12,8500,Services,EU
2024-11-06,13200,Subscriptions,NA
2024-11-18,4200,Add-ons,APAC
2024-12-02,17800,Services,NA
2024-12-15,9600,Subscriptions,EU
2024-07-04,11000,Subscriptions,NA
2024-08-07,7300,Services,EU
2024-09-10,5600,Add-ons,APAC
2024-01-03,5000,Subscriptions,NA
2024-04-14,7200,Services,EU
"""
    )
    df = pd.read_csv(csv, parse_dates=["date"])

    qmap = {"Q1": (1, 3), "Q2": (4, 6), "Q3": (7, 9), "Q4": (10, 12)}
    if quarter not in qmap:
        return pd.DataFrame()

    m_start, m_end = qmap[quarter]
    mask = (df["date"].dt.year == year) & (df["date"].dt.month.between(m_start, m_end))
    return df.loc[mask].reset_index(drop=True)


def load_into_dataframe(data: pd.DataFrame) -> pd.DataFrame:
    """Here it's trivial (already a DataFrame), but you could clean/transform here."""
    return data.copy()


def analyze_data(df: pd.DataFrame) -> Dict[str, Any]:
    """
    Compute a few simple KPIs/aggregates for the quarter.
    - Total revenue
    - Count of transactions
    - Revenue by category
    - Revenue by month
    """
    if df.empty:
        return {"summary": "No data for requested period.", "kpis": {}, "by_category": {}, "by_month": {}}

    total = float(df["amount"].sum())
    n_tx = int(df.shape[0])
    by_category = df.groupby("category")["amount"].sum().sort_values(ascending=False).to_dict()
    by_month = (
        df.assign(month=df["date"].dt.to_period("M").astype(str))
          .groupby("month")["amount"].sum()
          .sort_index()
          .to_dict()
    )
    return {
        "summary": "Quarterly analytics computed.",
        "kpis": {"total_revenue": total, "transactions": n_tx, "avg_ticket": total / max(n_tx, 1)},
        "by_category": by_category,
        "by_month": by_month,
    }


def generate_plot(by_month: Dict[str, float], title: str = "Quarterly Revenue by Month",
                  outfile: str = "quarterly_plot.png") -> str:
    """
    Create a simple line chart with matplotlib (one chart, no custom colors).
    Returns the saved file path.
    """
    if not by_month:
        return ""

    months = list(by_month.keys())
    values = list(by_month.values())

    plt.figure()               # one figure, one plot (no subplots)
    plt.plot(months, values, marker="o")
    plt.title(title)
    plt.xlabel("Month")
    plt.ylabel("Revenue")
    plt.tight_layout()
    plt.savefig(outfile, dpi=150)
    plt.close()
    return outfile


def create_report(period_label: str, kpis: Dict[str, Any], by_category: Dict[str, float],
                  plot_path: Optional[str], outfile: str = "quarterly_report.md") -> str:
    """
    Write a simple Markdown report that includes KPIs, a category table, and references the plot.
    """
    md = []
    md.append(f"# Quarterly Report — {period_label}\n")
    if not kpis:
        md.append("_No data available for this period._\n")
    else:
        md.append("## Key Metrics\n")
        md.append(f"- **Total Revenue:** {kpis['total_revenue']:.2f}")
        md.append(f"- **Transactions:** {kpis['transactions']}")
        md.append(f"- **Average Ticket:** {kpis['avg_ticket']:.2f}\n")

        if by_category:
            md.append("## Revenue by Category\n")
            md.append("| Category | Revenue |\n|---|---:|")
            for cat, amt in by_category.items():
                md.append(f"| {cat} | {amt:.2f} |")
            md.append("")

    if plot_path and Path(plot_path).exists():
        md.append("## Monthly Trend\n")
        md.append(f"![Quarterly Plot]({plot_path})\n")

    Path(outfile).write_text("\n".join(md), encoding="utf-8")
    return outfile

# ----------------------------- Agents ----------------------------------------
class DatabaseAgent:
    def run(self, quarter: str, year: int) -> TaskResult:
        df = query_database(quarter, year)
        if df.empty:
            return TaskResult(ok=False, info=f"No rows for {quarter} {year}", data=df)
        return TaskResult(ok=True, info=f"Fetched {len(df)} rows.", data=df)


class AnalysisAgent:
    def run(self, df: pd.DataFrame) -> TaskResult:
        try:
            clean_df = load_into_dataframe(df)
            summary = analyze_data(clean_df)
            return TaskResult(ok=True, info=summary["summary"], data=summary)
        except Exception as e:
            return TaskResult(ok=False, info=f"Analysis error: {e}")


class GraphAgent:
    def run(self, by_month: Dict[str, float], period_label: str) -> TaskResult:
        try:
            if not by_month:
                return TaskResult(ok=False, info="No monthly data to plot.")
            plot_path = generate_plot(by_month, title=f"Revenue by Month — {period_label}")
            return TaskResult(ok=True, info="Plot created.", artifacts={"plot": plot_path})
        except Exception as e:
            return TaskResult(ok=False, info=f"Plot error: {e}")


class ReportAgent:
    def run(self, period_label: str, kpis: Dict[str, Any], by_category: Dict[str, float],
            plot_path: Optional[str]) -> TaskResult:
        try:
            report = create_report(period_label, kpis, by_category, plot_path)
            return TaskResult(ok=True, info="Report created.", artifacts={"report": report})
        except Exception as e:
            return TaskResult(ok=False, info=f"Report error: {e}")

# ----------------------------- Orchestrator -----------------------------------
class OrchestratorAgent:
    QUARTER_REGEX = re.compile(r"\bQ([1-4])\b", re.IGNORECASE)

    def __init__(self):
        self.db_agent = DatabaseAgent()
        self.analysis_agent = AnalysisAgent()
        self.graph_agent = GraphAgent()
        self.report_agent = ReportAgent()

    def _parse_period(self, text: str) -> Tuple[str, int, str]:
        """
        Extract quarter (Q1..Q4) and year from user text.
        Returns (quarter, year, label). Falls back gracefully if missing.
        """
        t = text.strip()
        # Try to find Qx
        m = self.QUARTER_REGEX.search(t)
        quarter = f"Q{m.group(1)}" if m else "Q4"   # default example
        # Try to find a year-like token
        y = None
        for tok in re.findall(r"\b(20\d{2})\b", t):
            try:
                y = int(tok); break
            except:  # pragma: no cover
                pass
        year = y if y else dtparse("today").year
        label = f"{quarter} {year}"
        return quarter.upper(), year, label

    def handle(self, user_prompt: str) -> TaskResult:
        quarter, year, label = self._parse_period(user_prompt)

        # 1) Database
        db_res = self.db_agent.run(quarter, year)
        if not db_res.ok:
            return TaskResult(ok=False, info=f"[DB] {db_res.info}")

        # 2) Analysis
        an_res = self.analysis_agent.run(db_res.data)
        if not an_res.ok:
            return TaskResult(ok=False, info=f"[Analysis] {an_res.info}")

        summary = an_res.data
        kpis = summary.get("kpis", {})
        by_category = summary.get("by_category", {})
        by_month = summary.get("by_month", {})

        # 3) Graph
        gr_res = self.graph_agent.run(by_month, period_label=label)
        plot_path = None
        if gr_res.ok and gr_res.artifacts:
            plot_path = gr_res.artifacts.get("plot")

        # 4) Report
        rep_res = self.report_agent.run(label, kpis, by_category, plot_path)
        if not rep_res.ok:
            return TaskResult(ok=False, info=f"[Report] {rep_res.info}")

        # Final bundle
        final_artifacts = {"report": rep_res.artifacts.get("report")}
        if plot_path:
            final_artifacts["plot"] = plot_path

        return TaskResult(
            ok=True,
            info=f"Completed quarterly report for {label}.",
            data={"kpis": kpis, "by_category": by_category, "by_month": by_month},
            artifacts=final_artifacts
        )

# ------------------------------ Demo ------------------------------------------
if __name__ == "__main__":
    user_prompt = "Generate a quarterly report from the database for Q4 2024."
    orchestrator = OrchestratorAgent()
    result = orchestrator.handle(user_prompt)

    print("\nAgentic Workflow Result")
    print("=" * 80)
    print("Status:", "OK" if result.ok else "FAILED")
    print("Info  :", result.info)

    if result.data:
        print("\nKPIs:", result.data.get("kpis"))
        print("By Category:", result.data.get("by_category"))
        print("By Month:", result.data.get("by_month"))

    if result.artifacts:
        print("\nArtifacts:")
        for k, v in result.artifacts.items():
            print(f" - {k}: {v}")



Agentic Workflow Result
Status: OK
Info  : Completed quarterly report for Q4 2024.

KPIs: {'total_revenue': 65300.0, 'transactions': 6, 'avg_ticket': 10883.333333333334}
By Category: {'Subscriptions': 34800, 'Services': 26300, 'Add-ons': 4200}
By Month: {'2024-10': 20500, '2024-11': 17400, '2024-12': 27400}

Artifacts:
 - report: quarterly_report.md
 - plot: quarterly_plot.png
